In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
import os
from tensorflow.keras.applications import resnet
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

In [ ]:
from tensorflow import keras


In [ ]:
from google.colab import drive
import zipfile
drive.mount('/content/gdrive')

In [ ]:
dir_path = '/content/gdrive/MyDrive/FoodSegmentation/images.zip'
zip_ref = zipfile.ZipFile(dir_path, 'r')
zip_ref.extractall("/tmp")
zip_ref.close()


In [ ]:
!ls '/tmp/images'

In [ ]:
IMAGE_SIZE = 100
CURRENT_PATH = "/tmp/images"
TRAIN_PATH = CURRENT_PATH + "/images/train"
TEST_PATH = CURRENT_PATH + "/images/test"
VALIDATION_PATH = CURRENT_PATH + "/images/validation"
BATCH_SIZE = 64
EPOCH_SIZE = 150
NAME = "Epoch size:150 Batch size:64 image_size:100px for 100 train images"


In [ ]:
def plot_history(history):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Training and Validation Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()

In [ ]:
def generate_pairs(images, labels):
    x_pairs = []
    y_pairs = []
    for i in range(len(images)):
        label = labels[i]
        j = np.random.choice(np.where(labels == label)[0])
        x_pairs.append([images[i], images[j]])
        y_pairs.append([1])
    
        k = np.random.choice(np.where(labels != label)[0])
        x_pairs.append([images[i], images[k]])
        y_pairs.append([0])

    indices = np.arange(len(x_pairs))
    np.random.shuffle(indices)
    print("pairs length:",len(x_pairs))
    return np.array(x_pairs)[indices], np.array(y_pairs)[indices]


In [ ]:
def norm(features):
    return tf.norm(features[0] - features[1], ord='euclidean', axis=1, keepdims=True)


In [ ]:
def accuracy(y_true, y_pred):
    return metrics.binary_accuracy(y_true, 1 - y_pred)


In [ ]:
def siamese_network():
    input1 = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    input2 = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    twin = resnet.ResNet50( 
        weights="imagenet", input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, pooling='avg',
    )
    distance = layers.Lambda(norm)([
        twin(input1), 
        twin(input2)
    ])
    return Model(inputs=[input1, input2], outputs=distance)     


In [ ]:
def images_to_array(image_start,image_end):
 
    dataset_array = []
    dataset_labels = []
    class_counter = 0
    image_counter = 0
    classes_names = os.listdir(CURRENT_PATH)
    for current_class_name in classes_names:
        if class_counter < 101:
            class_dir = os.path.join(CURRENT_PATH, current_class_name)
            if not ".DS_Store" in class_dir:
              images_in_class = os.listdir(class_dir)
              for image_file in images_in_class:
                  if image_file.endswith(".jpg") and image_counter >= image_start and image_counter <= image_end:
                      image_file_dir = os.path.join(class_dir, image_file)
                      img = tf.keras.preprocessing.image.load_img(image_file_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE))
                      img_array = tf.keras.preprocessing.image.img_to_array(img)
                      img_array = img_array/255.0
                      dataset_array.append(img_array)
                      dataset_labels.append(class_counter)
                  image_counter += 1
              class_counter += 1
              image_counter = 0
    dataset_array = np.array(dataset_array)
    dataset_labels = np.array(dataset_labels)
    return dataset_array, dataset_labels


In [ ]:
model = siamese_network()

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer="adam", 
    metrics=[accuracy]
)

In [ ]:
model.summary()


In [ ]:
train_dataset_array, train_dataset_array_labels = images_to_array(0,149)
test_dataset_array, test_dataset_array_labels = images_to_array(150,164)
images_pairs_train, labels_pairs_train = generate_pairs(train_dataset_array, train_dataset_array_labels)
images_pairs_test, labels_pairs_test = generate_pairs(test_dataset_array, test_dataset_array_labels)

In [ ]:
history = model.fit(
    x=[images_pairs_train[:, 0], images_pairs_train[:, 1]], 
    y=labels_pairs_train[:], 
    batch_size=BATCH_SIZE,
    epochs=EPOCH_SIZE,
    validation_split=0.3
)

In [ ]:
plot_history(history.history)

In [ ]:
model.save('600_images_150_epoch_binarycross.h5')

In [ ]:
model = keras.models.load_model("600_images_150_epoch_binarycross.h5")


In [ ]:
predictions = np.round(1 - model.predict([images_pairs_test[:, 0], images_pairs_test[:, 1]]))
accuracy = metrics.BinaryAccuracy()
accuracy.update_state(labels_pairs_test, predictions)
print(f"\nAccuracy: {accuracy.result().numpy()}")

In [ ]:
img_path_one = CURRENT_PATH + "/apple_pie/3915901.jpg"
img_path_two = CURRENT_PATH + "/apple_pie/3917257.jpg"

img_one = tf.io.read_file(img_path_one)
img_one = tf.image.convert_image_dtype(tf.io.decode_png(img_one, channels=3), dtype='float32')  # * 1./255
img_one = tf.image.resize(img_one, (IMAGE_SIZE, IMAGE_SIZE), method=tf.image.ResizeMethod.BILINEAR)
img_one_final = tf.expand_dims(img_one, 0)

img_two = tf.io.read_file(img_path_two)
img_two = tf.image.convert_image_dtype(tf.io.decode_png(img_two, channels=3), dtype='float32')  # * 1./255
img_two = tf.image.resize(img_two, (IMAGE_SIZE, IMAGE_SIZE), method=tf.image.ResizeMethod.BILINEAR)
img_two_final = tf.expand_dims(img_two, 0)

In [ ]:
true_lab = 0

y_pred= model.predict([img_one_final, img_two_final])
print(y_pred)
if y_pred < 0.8:
  print("Similar")
else:
  print("Not similar")